## Package

In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import FashionMNIST

from torch.utils.data import DataLoader

from architectures_28x28.CKAN_BN import CKAN_BN
from architectures_28x28.SimpleModels import *
from architectures_28x28.ConvNet import ConvNet
from architectures_28x28.KANConvs_MLP import KANC_MLP
from architectures_28x28.KKAN import KKAN_Convolutional_Network
from architectures_28x28.conv_and_kan import NormalConvsKAN


## Dataset

In [2]:
# Transformaciones
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
fashion_mnist_test = FashionMNIST(root='./data', train=False, transform=transform, download=True)
fashion_mnist_test_loader = DataLoader(fashion_mnist_test, batch_size=64, shuffle=False)

## Set Device

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Load model

In [4]:
def load_model(model_class, model_path):
    model = model_class()
    model.to(device)
    model.load_state_dict(torch.load(model_path, weights_only=True, map_location=device))
    model.eval()
    return model

In [8]:
def calculate_metrics(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for data, target in dataloader:

            if not isinstance(data, torch.Tensor):
                data = torch.tensor(data)
            if not isinstance(target, torch.Tensor):
                target = torch.tensor(target)


            data, target = data.to(device), target.to(device)  # Ensure data and target are on the correct device
            output = model(data)
            preds = output.argmax(dim=1, keepdim=True).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(target.cpu().numpy())
    
    all_preds = np.array(all_preds).flatten()
    all_labels = np.array(all_labels).flatten()
    
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    return accuracy, precision, recall, f1



def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [6]:
models = {
    'SimpleCNN_2': 'models/fashion-mnist/SimpleCNN_2/best_model.pth',
    'SimpleCNN': 'models/fashion-mnist/SimpleCNN/best_model.pth',
    'SimpleLinear': 'models/fashion-mnist/SimpleLinear/best_model.pth',
    'ConvNet': 'models/fashion-mnist/ConvNet/best_model.pth',
    'NormalConvsKAN': 'models/fashion-mnist/NormalConvsKAN/best_model.pth',
}


In [9]:
fashion_mnist_results = []

for model_name, model_path in models.items():
    model_class = globals()[model_name]
    model = load_model(model_class, model_path)
    
    fashion_mnist_metrics = calculate_metrics(model, fashion_mnist_test_loader, device)
    num_params = count_parameters(model)
    
   
    fashion_mnist_results.append({
        'Model': model_name,
        'Accuracy': fashion_mnist_metrics[0],
        'Precision': fashion_mnist_metrics[1],
        'Recall': fashion_mnist_metrics[2],
        'F1 Score': fashion_mnist_metrics[3],
        'Parameters': num_params
    })

In [13]:
kkan_mlp = KANC_MLP(device=device)
kkan_mlp.to(device)
kkan_mlp.load_state_dict(torch.load('models/fashion-mnist/KANC_MLP/best_model.pth',weights_only=True))
kkan_mlp.eval()  


kkan_mlp_metrics = calculate_metrics(kkan_mlp, fashion_mnist_test_loader, device)
num_params_kkan_mlp = count_parameters(kkan_mlp)

fashion_mnist_results.append({
        'Model': "KANC_MLP",
        'Accuracy': kkan_mlp_metrics[0],
        'Precision': kkan_mlp_metrics[1],
        'Recall': kkan_mlp_metrics[2],
        'F1 Score': kkan_mlp_metrics[3],
        'Parameters': num_params_kkan_mlp
    })


In [11]:
kkan_model = KKAN_Convolutional_Network()
# kkan_model.to(device)
kkan_model.load_state_dict(torch.load('models/fashion-mnist/KKAN_Convolutional/best_model.pth',weights_only=True,  map_location=torch.device('cpu')))
kkan_model.eval()  

KKAN_Convolutional_Network(
  (conv1): KAN_Convolutional_Layer(
    (convs): ModuleList(
      (0-4): 5 x KAN_Convolution(
        (conv): KANLinear(
          (base_activation): SiLU()
        )
      )
    )
  )
  (conv2): KAN_Convolutional_Layer(
    (convs): ModuleList(
      (0-4): 5 x KAN_Convolution(
        (conv): KANLinear(
          (base_activation): SiLU()
        )
      )
    )
  )
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (flat): Flatten(start_dim=1, end_dim=-1)
  (kan1): KANLinear(
    (base_activation): SiLU()
  )
)

In [12]:
# 
def calculate_metrics_cpu(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for data, target in dataloader:
            # data, target = data.to(device), target.to(device)  # Ensure data and target are on the correct device
            output = model(data)
            preds = output.argmax(dim=1, keepdim=True).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(target.cpu().numpy())
    
    all_preds = np.array(all_preds).flatten()
    all_labels = np.array(all_labels).flatten()
    
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    return accuracy, precision, recall, f1

kkan_metrics = calculate_metrics_cpu(kkan_model, fashion_mnist_test_loader)
num_params_kkan = count_parameters(kkan_model)

print(kkan_metrics)

fashion_mnist_results.append({
        'Model': "KKAN_Convolutional_Network",
        'Accuracy': kkan_metrics[0],
        'Precision': kkan_metrics[1],
        'Recall': kkan_metrics[2],
        'F1 Score': kkan_metrics[3],
        'Parameters': num_params
    })

(0.8901, 0.8896094174870072, 0.8901, 0.8897385508046677)


In [14]:
# Create DataFrames
df_fashion_mnist_results = pd.DataFrame(fashion_mnist_results)

# Style the DataFrames

df_fashion_mnist_styled = df_fashion_mnist_results.style.format({
    'Accuracy': '{:.4f}',
    'Precision': '{:.4f}',
    'Recall': '{:.4f}',
    'F1 Score': '{:.4f}',
    'Parameters': '{:,}'
}).set_caption("Model Evaluation Metrics on Fashion-MNIST Dataset")

# Display the styled DataFrames
df_fashion_mnist_styled

,Model,Accuracy,Precision,Recall,F1 Score,Parameters
0,SimpleCNN_2,0.8988,0.8981,0.8988,0.8981,"157,030"
1,SimpleCNN,0.8695,0.8692,0.8695,0.8691,"2,740"
2,SimpleLinear,0.8428,0.8412,0.8428,0.8412,"7,850"
3,ConvNet,0.9248,0.9243,0.9248,0.9243,"887,530"
4,NormalConvsKAN,0.8861,0.8853,0.8861,0.8852,"37,030"
5,KKAN_Convolutional_Network,0.8901,0.8896,0.8901,0.8897,"37,030"
6,KANC_MLP,0.8882,0.8880,0.8882,0.8880,"163,726"
